# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

# Hackathon: Voice commands based E-commerce ordering system
The goal of the hackathon is to train your model on different types of voice data (such as studio data and your own team data) and able to place order based on user preferences.

## Grading = 40 Marks

### **Objectives:**

Stage 0 - Obtain Features from Audio samples

Stage 1 (22 Marks) - Define and train a CNN model on Studio data and deploy the model in the server 

Stage 2 (18 Marks) - Collect your voice samples (team data) and refine the classifier trained on Studio_data. Deploy the model in the server.

## Dataset Description

The data contains voice samples of classes - Zero, One, Two, Three, Four, Five. Each class is denoted by a numerical label from 0 to 5.

The audio files collected in a Studio dataset contain very few noise samples and all the files are in wav format.

The audio files recorded for the studio are saved with the following naming convention: 

● Class Representation + user_id + sample_ID (or noise + sample_ID)

> For example: The voice sample by the user b2 recorded “Zero”, it is saved as 0_b2_35.wav. Here 35 is sample ID, 2 is the user id and ‘0’ is the label of that sample.




In [ ]:
! wget https://cdn.iiith.talentsprint.com/aiml/Hackathon_data/B17_studio_rev_data.zip
! unzip B17_studio_rev_data.zip

In [ ]:
import os
import sys
import glob
import torch
import librosa
import warnings
import numpy as np
import torch.nn as nn
from time import sleep
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
warnings.filterwarnings('ignore')

## **Stage 0:** Obtain Features from Audio samples
---

### Generate features from an audio sample of '.wav' format
- Code is available to extract the features

In [ ]:
# Caution: Do not change the default parameters
def get_features(filepath, sr=8000, n_mfcc=30, n_mels=128, frames = 15):
    # The following function contains code to produce features of the audio sample.  
    y, sr = librosa.load(filepath, sr=sr)
    D = np.abs(librosa.stft(y))**2
    S = librosa.feature.melspectrogram(S=D)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    log_S = librosa.power_to_db(S,ref=np.max)
    features = librosa.feature.mfcc(S=log_S, n_mfcc=n_mfcc)
    if features.shape[1] < frames :
        features = np.hstack((features, np.zeros((n_mfcc, frames - features.shape[1]))))
    elif features.shape[1] > frames:
        features = features[:, :frames]

    # Find 1st order delta_mfcc
    delta1_mfcc = librosa.feature.delta(features, order=1)

    # Find 2nd order delta_mfcc
    delta2_mfcc = librosa.feature.delta(features, order=2)

    # Stacking delta_mfcc features in sequence horizontally (column wise)
    features = np.hstack((delta1_mfcc.flatten(), delta2_mfcc.flatten()))

    # Increase the dimension by inserting an axis along second dimension
    features = features.flatten()[:,np.newaxis]
    
    # Convert the numpy.ndarray to a Tensor object
    features = Variable(torch.from_numpy(features)).float()
    return features

All the voice samples needed for training are present in the folder `"studio_data"`

In [ ]:
%ls

B17_studio_rev_data.zip  sample_data/  studio_data/


##**Stage 1**:  Define and train a CNN model on Studio data and deploy the model in the server

---


### a) Extract features of Studio data (4 Marks)

 Load 'Studio data' and extract mfcc features

 **Evaluation Criteria:**

 * Complete the code in the load_data function
 * The function should take path of the folder containing audio samples as input
 * It should return features of all the audio samples present in the specified folder into single array (list of lists or 2-d numpy array) and their respective labels should be returned too

In [ ]:
def load_data(folder_path):
    features, labels = [], []
    for file in glob.glob(folder_path+"/*.wav", recursive=True):
        file_name = os.path.basename(file)
        label = (int((file_name.split("/")[-1]).split("_")[0]))
        feat = get_features(file)
        features.append(feat.numpy())
        labels.append(label)
    return features, labels

Load data from studio_data folder for extracting all features and labels

In [ ]:
studio_recorded_features, studio_recorded_labels = load_data('/content/studio_data')

In [ ]:
numOfClasses = len(set(studio_recorded_labels))
numOfClasses

6

In [ ]:
len(studio_recorded_features)

3979

Use train_test_split for splitting the train and test data

In [ ]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train_studio, X_test_studio, y_train_studio, y_test_studio = train_test_split(studio_recorded_features, studio_recorded_labels, test_size = 0.2, random_state=42, shuffle=False)

Load the dataset with DataLoader
- Refer to [torch.utils.data.TensorDataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset)
- Refer to [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

In [ ]:
batch_size = 28

# The TensorDataset takes an arbitrary number of input tensors.
train_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_train_studio), torch.LongTensor(y_train_studio))
test_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_test_studio), torch.LongTensor(y_test_studio))

# Loading the train dataset
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle = True)

In [ ]:
# Check the Input Shape
inputs, labels = next(iter(train_loader))
print(inputs.shape)

torch.Size([28, 900, 1])


### b) Define your CNN architecture (4 Marks)

[Hint](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)

In [ ]:
## Define your Architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Sample Convolution Layer 1 
        self.conv1 = nn.Conv1d(in_channels=900, out_channels=450, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(450)
        self.relu1 = nn.ReLU()

        # Sample Maxpool for the Convolutional Layer 1
        self.maxpool1 = nn.MaxPool1d(1)

        # Sample Dropout Layer
        self.dropout = nn.Dropout(p=0.25)

        # YOUR CODE HERE for defining more number of Convolutional layers with Maxpool as required (Hint: Use at least 2 more convolutional layers for better performance)
        self.conv2 = nn.Conv1d(in_channels=450, out_channels=225,kernel_size=1)
        self.bn2 = nn.BatchNorm1d(225)
        self.maxpool2 = nn.MaxPool1d(1)

        self.conv3 = nn.Conv1d(in_channels=225, out_channels=100,kernel_size=1)
        self.bn3 = nn.BatchNorm1d(100)
        self.maxpool3 = nn.MaxPool1d(1)

        # YOUR CODE HERE for defining the Fully Connected Layer and also define LogSoftmax
        self.fc1 = nn.Linear(100, 50)
        self.fc2 = nn.Linear(50, 25)
        self.fc3 = nn.Linear(25,6)

    def forward(self, x):
        # Convolution Layer 1, Maxpool and Dropout
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.dropout(out)

        # YOUR CODE HERE for the Convolutional Layers and Maxpool based on the defined Convolutional layers
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu1(out)
        out = self.maxpool2(out)

        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu1(out)
        out = self.maxpool3(out)

        out = out.view(-1,100)

        # YOUR CODE HERE for flattening the output of the final pooling layer to a vector. Flattening is simply arranging the 3D volume of numbers into a 1D vector
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)

        # YOUR CODE HERE for returning the output of LogSoftmax after applying Fully Connected Layer
        return F.log_softmax(out)

In [ ]:
## Define your CNN Architecture
class Net_1(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Sample Convolution Layer 1 
        self.conv1 = nn.Conv1d(in_channels=900, out_channels=400, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(400)
        self.relu1 = nn.ReLU()

        # Sample Maxpool for the Convolutional Layer 1
        self.maxpool1 = nn.MaxPool1d(1)

        # Sample Dropout Layer
        self.dropout = nn.Dropout(p=0.25)

        # YOUR CODE HERE for defining more number of Convolutional layers with Maxpool as required (Hint: Use at least 2 more convolutional layers for better performance)
        self.conv2 = nn.Conv1d(400, 200, 1)
        self.bn2 = nn.BatchNorm1d(200)
        
        self.conv3 = nn.Conv1d(200, 128, 1)
        self.bn3 = nn.BatchNorm1d(128)
        
        # YOUR CODE HERE for defining the Fully Connected Layer and also define LogSoftmax
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32, numOfClasses)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Convolution Layer 1, Maxpool and Dropout
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.dropout(out)
        # YOUR CODE HERE for the Convolutional Layers and Maxpool based on the defined Convolutional layers
        out = self.maxpool1(F.relu(self.bn2(self.conv2(out))))
        out = self.maxpool1(F.relu(self.bn3(self.conv3(out))))

        # YOUR CODE HERE for flattening the output of the final pooling layer to a vector. Flattening is simply arranging the 3D volume of numbers into a 1D vector
        x = out.view(-1, 128)
        x = self.fc1(x)
        x = self.fc2(x)
        
        # YOUR CODE HERE for returning the output of LogSoftmax after applying Fully Connected Layer
        return self.logsoftmax(x)

In [ ]:
# To run the training on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
model = Net()
model = model.to(device)
print(model)

criterion = nn.NLLLoss() # YOUR CODE HERE : Explore and declare loss function

optimizer = torch.optim.Adam(model.parameters(), lr=0.017) # YOUR CODE HERE : Explore on the optimizer and define with the learning rate

Net(
  (conv1): Conv1d(900, 450, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(450, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (maxpool1): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (conv2): Conv1d(450, 225, kernel_size=(1,), stride=(1,))
  (bn2): BatchNorm1d(225, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(225, 100, kernel_size=(1,), stride=(1,))
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=6, bias=True)
)


### c) Train and classify on the studio_data (3 Marks)

The goal here is to train the Model on voice samples collected in studio data and validate it continuously to calculate the loss and accuracy for the train dataset across each epoch.

Iterate over images in the train_loader and perform the following steps. 

1. First, zero out the gradients using zero_grad()

2. Pass the data to the model. Convert the data to GPU before passing data  to the model

3. Calculate the loss using a Loss function

4. Perform Backward pass using backward() to update the weights

5. Optimize and predict by using the torch.max()

6. Calculate the accuracy of the train dataset


In [ ]:
# YOUR CODE HERE. This will take time

# Record loss and accuracy of the train dataset
def train_model(num_epochs, model, train_loader, train_set):
    model.train()
    #Define the lists to store the results of loss and accuracy
    train_loss, train_accuracy = [], []
    for epoch in range(num_epochs):
        correct, iter_loss = 0, 0.0
        for (inputs, labels) in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            iter_loss += loss.item()
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum()
        train_loss.append(iter_loss/len(train_set))
        train_accuracy.append((100 * correct / len(train_set)))
        print('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}'.format(epoch+1, num_epochs, train_loss[-1], train_accuracy[-1]))
    return model

trained_model = train_model(20, model, train_loader, train_set)

Epoch 1/20, Training Loss: 0.056, Training Accuracy: 31.448
Epoch 2/20, Training Loss: 0.043, Training Accuracy: 49.042
Epoch 3/20, Training Loss: 0.036, Training Accuracy: 59.315
Epoch 4/20, Training Loss: 0.033, Training Accuracy: 64.499
Epoch 5/20, Training Loss: 0.030, Training Accuracy: 67.044
Epoch 6/20, Training Loss: 0.028, Training Accuracy: 71.568
Epoch 7/20, Training Loss: 0.026, Training Accuracy: 73.924
Epoch 8/20, Training Loss: 0.025, Training Accuracy: 75.620
Epoch 9/20, Training Loss: 0.024, Training Accuracy: 76.437
Epoch 10/20, Training Loss: 0.023, Training Accuracy: 77.788
Epoch 11/20, Training Loss: 0.021, Training Accuracy: 79.265
Epoch 12/20, Training Loss: 0.020, Training Accuracy: 80.584
Epoch 13/20, Training Loss: 0.018, Training Accuracy: 81.967
Epoch 14/20, Training Loss: 0.019, Training Accuracy: 81.904
Epoch 15/20, Training Loss: 0.018, Training Accuracy: 82.312
Epoch 16/20, Training Loss: 0.017, Training Accuracy: 83.632
Epoch 17/20, Training Loss: 0.016

### d) Testing Evaluation for CNN model (3 Marks)

Evaluate model with the given test data

1. Transform and load the test images.

2. Pass the test data through the model (network) to get the outputs

3. Get the predictions from a maximum value using torch.max

4. Compare with the actual labels and get the count of the correct labels

5. Calculate the accuracy based on the count of correct labels

### **Expected testing accuracy is above 80%**

In [ ]:
# YOUR CODE HERE to test the model
def test_model(model, test_loader, test_set):
    model.eval()
    test_accuracy = 0
    test_labels, test_predictions = [], []
    for inputs, labels in test_loader:
      # Convert the images and labels to Pytorch tensor 
      inputs = inputs.to(device)
      labels = labels.to(device)

      # Passing images to the model, which return the probabilites as outputs
      output = model(inputs) 
      
      # Picking the class/label with maximum probability                                                                 
      _, predicted = torch.max(output, 1)
      test_accuracy += (predicted == labels).sum().item()
      test_labels.extend(labels)
      test_predictions.extend(predicted)
    accuracy = 100 * (test_accuracy/(len(test_set)))
    print("Accuracy of Test Data is", accuracy)
    return [i.item() for i in test_labels], [i.item() for i in test_predictions]

test_labels, test_predictions = test_model(trained_model, test_loader, test_set)

Accuracy of Test Data is 82.28643216080403


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, test_predictions))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85       129
           1       0.82      0.77      0.80       147
           2       0.73      0.75      0.74       107
           3       0.93      0.89      0.91       131
           4       0.82      0.88      0.85       144
           5       0.79      0.78      0.79       138

    accuracy                           0.82       796
   macro avg       0.82      0.82      0.82       796
weighted avg       0.82      0.82      0.82       796



### e) Save and download your model (2 Marks)

**Save your model trained on studio data**

* Save the state dictionary of the classifier (use pytorch only), It will be useful in
integrating model to the web application

 [Hint](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

In [ ]:
### YOUR CODE HERE for saving the CNN model
state = {'net_dict': trained_model.state_dict()}
torch.save(state, 'studio_model.t7')

Download your trained model using the code below
* Give the path of model file to download through the browser

In [ ]:
from google.colab import files
files.download('studio_model.t7')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### f) Deploy and evaluate your model trained on Studio Data in the server (6 Marks).

(This can be done on the day of the Hackathon once the login username and password provided by the mentors in the lab) 

Deploy your model on the server, check the hackathon document (2-Server Access and File transfer For Voice based e-commerce ordering.pdf) for details. 

To order product in user interface, go through the document (3-Hackathon_II Application Interface Documentation.pdf) for details.


**Evaluation Criteria: Four consecutive utterances should be predicted correctly by the model**

- There are two stages in the e-commerce ordering application    
    - Ordering Product
    - Selecting the e-commerce platform
- If both the stages are cleared as per the evaluation criteria you will get
complete marks Otherwise, you will see a reduction in the marks

## **Stage 2:** Collect your voice samples and refine the classifier trained on studio_data and Team_data
---

### a) Collect your Team Voice Samples and extract features (6 Marks)

(This can be done on the day of the Hackathon once the login username and password is given by mentors in the lab)

* In order to collect the team data, ensure the server is active (2-Server Access and File transfer For Voice based e-commerce ordering.pdf)

* Refer document "3-Hackathon_II Application Interface Documentation.pdf" for collecting your team voice samples. These will get stored in your server

**Evaluation Criteria:**
* Load 'Team_data' and extract features
* Combine features of team data with the extracted features of studio data
* Split the combined features into train and test data
* Load the dataset with DataLoader

In [ ]:
!mkdir team_data

In [ ]:
# Replace <YOUR_GROUP_ID> with your Username given in the lab
!wget -qq -r -A .wav https://aiml-sandbox1.talentsprint.com/audio_recorder/b18h2test03/team_data/ -nH --cut-dirs=100  -P ./team_data

In [ ]:
# YOUR CODE HERE to Load data from teamdata folder for extracting all features and labels
team_recorded_features, team_recorded_labels = load_data('/content/team_data')

In [ ]:
# Combine the features of all voice samples (studio_data and teamdata)
# YOUR CODE HERE
features = team_recorded_features + studio_recorded_features 
labels = team_recorded_labels + studio_recorded_labels
len(features), len(labels)

(3979, 3979)

In [ ]:
# YOUR CODE HERE to split the combined features into train and test data (Hint: Use train_test_split)
X_train_studio_team, X_test_studio_team, y_train_studio_team, y_test_studio_team = train_test_split(features, labels, test_size = 0.2, random_state=42, shuffle=False)

In [ ]:
# YOUR CODE HERE to load the dataset with DataLoader

batch_size = 500

# The TensorDataset takes an arbitrary number of input tensors.
train_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_train_studio_team), torch.LongTensor(y_train_studio_team))
test_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_test_studio_team), torch.LongTensor(y_test_studio_team))

# Loading the train dataset
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle = True)

### b) Classify and download the model (6 Marks)

The goal here is to train and test your model on all voice samples collected in studio and team data

**Evaluation Criteria:**
* Refine your classifier (if needed)
* Train your model on the extracted train data
* Test your model on the extracted test data
* Save and download the trained model

### **Expected testing accuracy is above 80%**

In [ ]:
# YOUR CODE HERE for refining your classifier (if needed)

In [ ]:
# YOUR CODE HERE to train your model

# Record loss and accuracy of the train dataset
trained_model = train_model(20, model, train_loader, train_set)

Epoch 1/20, Training Loss: 0.001, Training Accuracy: 92.837
Epoch 2/20, Training Loss: 0.000, Training Accuracy: 93.277
Epoch 3/20, Training Loss: 0.000, Training Accuracy: 94.314
Epoch 4/20, Training Loss: 0.000, Training Accuracy: 95.256
Epoch 5/20, Training Loss: 0.000, Training Accuracy: 95.319
Epoch 6/20, Training Loss: 0.000, Training Accuracy: 95.884
Epoch 7/20, Training Loss: 0.000, Training Accuracy: 96.513
Epoch 8/20, Training Loss: 0.000, Training Accuracy: 96.073
Epoch 9/20, Training Loss: 0.000, Training Accuracy: 96.701
Epoch 10/20, Training Loss: 0.000, Training Accuracy: 96.890
Epoch 11/20, Training Loss: 0.000, Training Accuracy: 96.890
Epoch 12/20, Training Loss: 0.000, Training Accuracy: 96.827
Epoch 13/20, Training Loss: 0.000, Training Accuracy: 97.110
Epoch 14/20, Training Loss: 0.000, Training Accuracy: 97.424
Epoch 15/20, Training Loss: 0.000, Training Accuracy: 97.864
Epoch 16/20, Training Loss: 0.000, Training Accuracy: 97.330
Epoch 17/20, Training Loss: 0.000

In [ ]:
# YOUR CODE HERE to test your model
test_labels, test_predictions = test_model(trained_model, test_loader, test_set)

Accuracy of Test Data is 84.04522613065326


In [ ]:
print(classification_report(test_labels, test_predictions))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88       129
           1       0.87      0.70      0.77       147
           2       0.76      0.81      0.79       107
           3       0.93      0.92      0.92       131
           4       0.82      0.89      0.85       144
           5       0.80      0.86      0.83       138

    accuracy                           0.84       796
   macro avg       0.84      0.84      0.84       796
weighted avg       0.84      0.84      0.84       796



**Save your trained model**

* Save the state dictionary of the classifier (use pytorch only), It will be useful in
integrating model to the web application

 [Hint](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

In [ ]:
### YOUR CODE HERE for saving the CNN model
state = {'net_dict': trained_model.state_dict()}
torch.save(state, 'studio_team_model.t7')

Download your trained model using the code below
* Give the path of model file to download through the browser

In [ ]:
from google.colab import files
files.download('studio_team_model.t7')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### c) Deploy and evaluate your model trained on Studio Data + Team Data in the server (6 Marks).

(This can be done on the day of the Hackathon once the login username and password provided by the mentors in the lab) 

Deploy your model on the server, check the hackathon document (2-Server Access and File transfer For Voice based e-commerce ordering.pdf) for details. 

To order product in user interface, go through the document (3-Hackathon_II Application Interface Documentation.pdf) for details.


**Evaluation Criteria: Four consecutive utterances should be predicted correctly by the model**

- There are two stages in the e-commerce ordering application    
    - Ordering Product
    - Selecting the e-commerce platform
- If both the stages are cleared as per the evaluation criteria you will get
complete marks Otherwise, you will see a reduction in the marks

## **Testing in Colab**

In [ ]:
import random

def test_model(model_path, model_class, files_path):
    myModel = model_class
    ckpt = torch.load(model_path)
    myModel.load_state_dict(ckpt['net_dict'])
    myModel.to(device)
    myModel.eval()
    for i in range(5):
        testFile = random.choice(os.listdir(files_path))
        print("\n",i,"TESTING FILE:",testFile, end="\t")
        testSample = get_features(files_path+"/" + testFile)
        features = testSample.unsqueeze(0)
        predicted_label = model(features).argmax().item()
        print("Expected Output: {}\tPredicted Output: {}".format(int(testFile.split("_")[0]), predicted_label))

In [ ]:
test_model("studio_model.t7", Net(), "team_data")

IndexError: ignored

In [ ]:
test_model("studio_model.t7", Net(), "studio_data")

In [ ]:
test_model("studio_team_model.t7", Net(), "team_data")

In [ ]:
test_model("studio_team_model.t7", Net(), "studio_data")